<a href="https://colab.research.google.com/github/erikaduda/dsto-gan/blob/main/DSTO_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DSTO GAN**


Para melhorar o algoritmo de balanceamento DeepSMOTE com GAN e obter as melhores análises de F1-Score, Recall e Precision, vamos considerar algumas mudanças e melhorias:

1.Hiperparâmetros do Modelo : Ajusta os hiperparâmetros conforme a dimensão do espaço latente ( n_z), número de épocas ( epochs) e taxas de
Para melhorar o algoritmo de balanceamento DeepSMOTE com GAN e obter as melhores análises de F1-Score, Recall e Precision, vamos considerar algumas mudanças e melhorias:

1.Hiperparâmetros do Modelo : Ajusta os hiperparâmetros conforme a dimensão do espaço latente ( n_z), número de épocas ( epochs) e taxas de aprendizado ( lr) para obter melhor desempenho.

2.Arquitetura do Modelo : Experimentar com diferentes arquiteturas para o Encodere Decoder, como adicionar mais camadas ou utilizar diferentes funções de ativação.

3.Aumento de dados : Inclui técnicas de aumento de dados para aumentar a variedade dos dados sintéticos gerados.

4.Avaliação de Desempenho : Utilização diferentes classificados e fazer uma análise aprofundada das métricas para

Essa versão do código inclui um Discriminador para formar um GAN, o que deve melhorar a qualidade das amostras sintéticas geradas. O treinamento do GAN é realizado em cada época, alternando entre o treinamento do Discriminador e do Gerador. As amostras sintéticas são geradas pelo Decoder do GAN, o que deve ajudar a melhorar as métricas de avaliação dos classificados.



**Melhorias Implementadas:**

1.Arquitetura do GAN: Adicionada uma rede Discriminator para melhorar a qualidade das amostras sintéticas geradas pelo Decoder.

2.Função G_SM1: Agora usa o GAN para gerar amostras sintéticas.

3.Treinamento do GAN: Implementado o treinamento do GAN, ajustando o gerador e o discriminador.

4.Classificadores: Adicionado o XGBoost aos classificadores.

5.Pipeline de Validação Cruzada: Implementada validação cruzada estratificada e coleta de métricas.

6.Salvamento e Carregamento de Modelos:Ajustado o salvamento e carregamento dos modelos GAN e dos classificadores

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bayesian-optimization

In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import os
import torch.optim as optim
import joblib
from torch.utils.data import TensorDataset
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV


# Definir parâmetros para os classificadores
param_spaces = {
    'Decision Tree': {'max_depth': (3, 15)},
    'Random Forest': {'n_estimators': (10, 100)},
    'Neural Network': {'alpha': (1e-4, 1e-2, 'log-uniform')},
    'KNN': {'n_neighbors': (3, 10)},
    'XGBoost': {'learning_rate': (0.01, 0.3)}
}


# Definir argumentos para o modelo
args = {
    'dim_h': 64,
    'n_z': 10,
    'lr': 0.0002,
    'epochs': 100,
    'batch_size': 64,
    'save': True,
    'train': True
}

class Encoder(nn.Module):
    def __init__(self, args, num_input_features):
        super(Encoder, self).__init__()
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        self.fc1 = nn.Linear(num_input_features, self.dim_h)
        self.fc2 = nn.Linear(self.dim_h, self.dim_h)
        self.fc_mean = nn.Linear(self.dim_h, self.n_z)
        self.fc_logvar = nn.Linear(self.dim_h, self.n_z)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, args, num_input_features):
        super(Decoder, self).__init__()
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        self.fc1 = nn.Linear(self.n_z, self.dim_h)
        self.fc2 = nn.Linear(self.dim_h, self.dim_h)
        self.fc_output = nn.Linear(self.dim_h, num_input_features)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc_output(x)
        return x

class Discriminator(nn.Module):
    def __init__(self, num_input_features):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(num_input_features, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

def G_SM1(X, y, n_to_sample, cl, encoder, decoder):
    # Genera amostras sintéticas usando o GAN
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataloader = torch.utils.data.DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=args['batch_size'], shuffle=True)

    synthetic_data = []
    for _ in range(n_to_sample):
        z = torch.randn(1, args['n_z'])
        synthetic_sample = decoder(z).detach().numpy()
        synthetic_data.append(synthetic_sample)

    synthetic_data = np.vstack(synthetic_data)
    synthetic_labels = np.array([cl] * n_to_sample)
    return synthetic_data, synthetic_labels

def calculate_n_to_sample(y):
    class_counts = np.bincount(y)
    major_class_count = np.max(class_counts)
    n_classes = len(class_counts)
    n_to_sample_dict = {cl: major_class_count - class_counts[cl] for cl in range(n_classes)}
    return n_to_sample_dict, major_class_count

# Diretório de entrada e saída
INPUT_DIR = "/content/drive/MyDrive/PHD_new/dataset_tratado/fast/"
OUTPUT_DIR_TRAIN = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/treino/"
OUTPUT_DIR_TEST = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/teste/"
MODELO_DIR = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/modelos/"
MODELO_DIR_DST = "/content/drive/MyDrive/PHD_new/resultados/dsto_gan/modelos_dsto/"

# Verificar se o diretório de saída existe, se não, criar
os.makedirs(OUTPUT_DIR_TRAIN, exist_ok=True)
os.makedirs(OUTPUT_DIR_TEST, exist_ok=True)
os.makedirs(MODELO_DIR, exist_ok=True)
os.makedirs(MODELO_DIR_DST, exist_ok=True)

# Listar arquivos .csv no diretório de entrada
csv_files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.csv')]

# Iterar sobre os arquivos .csv
for csv_file in csv_files:
    input_path = os.path.join(INPUT_DIR, csv_file)
    data = pd.read_csv(input_path)
    X = data.drop('class', axis=1).values
    y = data['class'].values
    print(f"Processing: {csv_file}")
    print(f"CLASS DISTRIBUTION:\n{data['class'].value_counts(normalize=True) * 100}\n")

    # Dividir os dados em conjunto de treinamento e validação
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    # Inicializar o DataFrame para armazenar as métricas de avaliação
    reports_df_train = pd.DataFrame(columns=['Classifier', 'Class', 'Fold', 'Precision', 'Recall', 'F1-Score'])
    reports_df_val = pd.DataFrame(columns=['Classifier', 'Class', 'Precision', 'Recall', 'F1-Score'])


    # Define classifiers
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Neural Network': MLPClassifier(),
        'KNN': KNeighborsClassifier(),
        'XGBoost': XGBClassifier()  # Adicionando XGBClassifier
    }

    # Inicializar listas para armazenar as métricas de validação de cada classificador
    precision_val_list = {classifier_name: [] for classifier_name in classifiers}
    recall_val_list = {classifier_name: [] for classifier_name in classifiers}
    f1_val_list = {classifier_name: [] for classifier_name in classifiers}

    # Realizar validação cruzada estratificada de 10 folds
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    for fold, (train_index, test_index) in enumerate(skf.split(X_train, y_train), 1):
        X_fold_train, X_fold_test = X_train[train_index], X_train[test_index]
        y_fold_train, y_fold_test = y_train[train_index], y_train[test_index]

        # Treinar o GAN
        if args['train']:
            encoder = Encoder(args, X_fold_train.shape[1])
            decoder = Decoder(args, X_fold_train.shape[1])
            discriminator = Discriminator(X_fold_train.shape[1])
            optimizer_g = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=args['lr'])
            optimizer_d = optim.Adam(discriminator.parameters(), lr=args['lr'])
            criterion_g = nn.MSELoss()
            criterion_d = nn.BCELoss()

            dataloader = torch.utils.data.DataLoader(torch.tensor(X_fold_train, dtype=torch.float32), batch_size=args['batch_size'], shuffle=True)

            for epoch in range(args['epochs']):
                for batch in dataloader:
                    # Treinar o discriminador
                    optimizer_d.zero_grad()
                    real_labels = torch.ones(batch.size(0), 1)
                    fake_labels = torch.zeros(batch.size(0), 1)

                    outputs = discriminator(batch)
                    d_loss_real = criterion_d(outputs, real_labels)

                    z = torch.randn(batch.size(0), args['n_z'])
                    fake_data = decoder(z)
                    outputs = discriminator(fake_data.detach())
                    d_loss_fake = criterion_d(outputs, fake_labels)

                    d_loss = d_loss_real + d_loss_fake
                    d_loss.backward()
                    optimizer_d.step()

                    # Treinar o gerador
                    optimizer_g.zero_grad()
                    outputs = discriminator(fake_data)
                    g_loss = criterion_g(fake_data, batch) + criterion_d(outputs, real_labels)
                    g_loss.backward()
                    optimizer_g.step()

                if args['save']:
                    torch.save(encoder.state_dict(), os.path.join(MODELO_DIR_DST, f'encoder_epoch{epoch}.pt'))
                    torch.save(decoder.state_dict(), os.path.join(MODELO_DIR_DST, f'decoder_epoch{epoch}.pt'))

        encoder.load_state_dict(torch.load(os.path.join(MODELO_DIR_DST, 'encoder_epoch99.pt')))
        decoder.load_state_dict(torch.load(os.path.join(MODELO_DIR_DST, 'decoder_epoch99.pt')))

        y = y.astype(np.int64)
        n_to_sample_dict, major_class_count = calculate_n_to_sample(y)

        X_synthetic_list = []
        y_synthetic_list = []
        for cl, n_samples in n_to_sample_dict.items():
            if n_samples > 0:
                X_synthetic, y_synthetic = G_SM1(X, y, n_samples, cl, encoder, decoder)
                X_synthetic_list.append(X_synthetic)
                y_synthetic_list.append(y_synthetic)

        if X_synthetic_list:
            X_synthetic_combined = np.concatenate(X_synthetic_list, axis=0)
            y_synthetic_combined = np.concatenate(y_synthetic_list, axis=0)
            X_combined = np.vstack((X, X_synthetic_combined))
            y_combined = np.hstack((y, y_synthetic_combined))
        else:
            X_combined = X
            y_combined = y

        classifiers = {
            'Decision Tree': DecisionTreeClassifier(),
            'Random Forest': RandomForestClassifier(),
            'Neural Network': MLPClassifier(),
            'KNN': KNeighborsClassifier(),
            'XGBoost': XGBClassifier()
        }


        optimized_classifiers = {}
        for classifier_name, classifier in classifiers.items():
            search = BayesSearchCV(classifier, param_spaces[classifier_name], n_iter=10, cv=3, n_jobs=-1, random_state=42)
            search.fit(X_train, y_train)
            optimized_classifiers[classifier_name] = search.best_estimator_

        for classifier_name, classifier in optimized_classifiers.items():
            classifier.fit(X_combined, y_combined)
            joblib.dump(classifier, os.path.join(MODELO_DIR, f'{csv_file}_{classifier_name}_fold{fold}_model.pkl'))

            y_pred_train = classifier.predict(X_combined)
            precision_train = precision_score(y_combined, y_pred_train, average=None)
            recall_train = recall_score(y_combined, y_pred_train, average=None)
            f1_train = f1_score(y_combined, y_pred_train, average=None)

            for class_label, precision, recall, f1 in zip(np.unique(y_combined), precision_train, recall_train, f1_train):
                reports_df_train = pd.concat([reports_df_train, pd.DataFrame({
                    'Classifier': classifier_name,
                    'Class': class_label,
                    'Fold': fold,
                    'Precision': precision,
                    'Recall': recall,
                    'F1-Score': f1
                }, index=[0])])

            y_pred_val = classifier.predict(X_val)
            precision_val = precision_score(y_val, y_pred_val, average=None)
            recall_val = recall_score(y_val, y_pred_val, average=None)
            f1_val = f1_score(y_val, y_pred_val, average=None)

            precision_val_list[classifier_name].append(precision_val)
            recall_val_list[classifier_name].append(recall_val)
            f1_val_list[classifier_name].append(f1_val)

    for classifier_name in classifiers:
        precision_val_avg = np.mean(precision_val_list[classifier_name], axis=0)
        recall_val_avg = np.mean(recall_val_list[classifier_name], axis=0)
        f1_val_avg = np.mean(f1_val_list[classifier_name], axis=0)

        for class_label, precision, recall, f1 in zip(np.unique(y), precision_val_avg, recall_val_avg, f1_val_avg):
            reports_df_val = pd.concat([reports_df_val, pd.DataFrame({
                'Classifier': classifier_name,
                'Class': class_label,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1
            }, index=[0])])

    reports_df_train.to_csv(os.path.join(OUTPUT_DIR_TRAIN, f'{csv_file}'), index=False)
    reports_df_val.to_csv(os.path.join(OUTPUT_DIR_TEST, f'{csv_file}'), index=False)


Processing: IT_customer_churn.csv
CLASS DISTRIBUTION:
class
0    73.463013
1    26.536987
Name: proportion, dtype: float64



<ipython-input-6-c4a840ed8087>:218: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(os.path.join(MODELO_DIR_DST, 'encoder_epoch99.pt')))
<ip